Nama : Muhammad Alif Darmamulia

NIM : 10119059

Asal Universitas : Institut Teknologi Bandung

Mulai Pengerjaan : 09/11/2020

In [1]:
"""
TO-DO:
1. Observasi data & Cleaning data
2. Kelompokkan data berdasar tanggal
3. 
"""

'\nTO-DO:\n1. Observasi data & Cleaning data\n2. Kelompokkan data berdasar tanggal\n3. \n'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("covid_19_data.csv")
n = len(df)
df.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [3]:
df.tail()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
116800,116801,09/23/2020,Zaporizhia Oblast,Ukraine,2020-09-24 04:23:38,3149.0,49.0,1158.0
116801,116802,09/23/2020,Zeeland,Netherlands,2020-09-24 04:23:38,1270.0,72.0,0.0
116802,116803,09/23/2020,Zhejiang,Mainland China,2020-09-24 04:23:38,1282.0,1.0,1272.0
116803,116804,09/23/2020,Zhytomyr Oblast,Ukraine,2020-09-24 04:23:38,5191.0,92.0,2853.0
116804,116805,09/23/2020,Zuid-Holland,Netherlands,2020-09-24 04:23:38,29513.0,1372.0,0.0


Data Observation

In [4]:
column = ["SNo", "ObservationDate", "Province/State", "Country/Region", "Last Update", "Confirmed", "Deaths", "Recovered"]

for col in column:
    print(col,"=",df[col].isnull().sum(), end="\n")
print("Data Length :",len(df))

SNo = 0
ObservationDate = 0
Province/State = 35353
Country/Region = 0
Last Update = 0
Confirmed = 0
Deaths = 0
Recovered = 0
Data Length : 116805


Cleaning Data (Untuk 2 cell pertama)

In [5]:
df = df.drop(columns=["SNo", "Last Update"])
df = df.fillna("Undefinied")

In [6]:
df_update1 = df.groupby(["Country/Region", "ObservationDate"]).mean()
# df_update1.to_csv('data_filtered.csv') // Dijadikan komen karena telah tersimpan file datasetnya. Kecuali apabila filenya belum ada

In [7]:
# Mendeskripsikan summary data observasi secara keseluruhan 
df_update1.describe()

,Confirmed,Deaths,Recovered
count,38831.00000,38831.000000,38831.000000
mean,16447.37156,545.655610,10743.058545
std,48287.35734,1901.071354,37001.298678
min,0.00000,0.000000,0.000000
25%,138.00000,2.000000,25.000000
50%,1552.00000,31.000000,668.000000
75%,10149.00000,249.000000,5129.000000
max,665188.00000,31106.000000,594229.000000


In [8]:
df2 = pd.read_csv('data_filtered.csv')
df2.head()

,Country/Region,ObservationDate,Confirmed,Deaths,Recovered
0,Azerbaijan,02/28/2020,1.0,0.0,0.0
1,"('St. Martin',)",03/10/2020,2.0,0.0,0.0
2,Afghanistan,02/24/2020,1.0,0.0,0.0
3,Afghanistan,02/25/2020,1.0,0.0,0.0
4,Afghanistan,02/26/2020,1.0,0.0,0.0


In [9]:
from dateutil.parser import parse

last_ = int(df2["ObservationDate"][0].replace("/",""))
idx = 0
for i in range(len(df2)):
    last1_ = int(df2["ObservationDate"][i].replace("/",""))
    if last1_ > last_ :
        idx = i
        last_ = last1_    
# Mendapatkan tanggal final observasi
(parse(df2["ObservationDate"][idx]) - parse("09/09/2020"))

datetime.timedelta(days=14)

In [10]:
# Mengambil nama-nama negara yang diobservasi
arr_country = np.unique(df2["Country/Region"])

In [19]:
countor = len(df2)//len(arr_country)

set_country_name = [] # Set nama negara yang bersesuaian dengan koefisien korelasinya
set_dataFrame1 = [] # Set koefisien korelasi antara penambahan hari dengan peningkatan kasus positif
set_dataFrame2 = [] # Set koefisien korelasi antara penambahan hari dengan peningkatan kasus sembuh
set_dataFrame3 = [] # Set koefisien korelasi antara penambahan hari dengan peningkatan kasus meninggal

for sign in arr_country:
    arr_date_count = []
    arr_increase_case = []
    arr_healthy_case = []
    arr_death_case = []
    if len(df2.loc[df2["Country/Region"] == sign]) >= countor: # Hanya ditinjau pada negara-negara yang telah publikasi sebanyak >= 174 kasus
        dataset = df2.loc[df2["Country/Region"] == sign]
        set_country_name.append(sign)
        early_time = dataset["ObservationDate"].iloc[0]
        idx = 0
        for time in dataset["ObservationDate"]:
            time_delta = (parse(time) - parse(early_time)).days
            arr_date_count.append(time_delta)
            arr_increase_case.append(dataset["Confirmed"].iloc[idx])
            arr_healthy_case.append(dataset["Recovered"].iloc[idx])
            arr_death_case.append(dataset["Deaths"].iloc[idx])
            idx += 1
        corrcoef1 = np.corrcoef([arr_date_count, arr_increase_case])
        corrcoef2 = np.corrcoef([arr_date_count, arr_healthy_case])
        corrcoef3 = np.corrcoef([arr_date_count, arr_death_case])
        set_dataFrame1.append(corrcoef1[1][0])
        set_dataFrame2.append(corrcoef2[1][0])
        set_dataFrame3.append(corrcoef3[1][0])

Data = {"Country" : set_country_name, "Koefisien Korelasi Penambahan Kasus" : set_dataFrame1
       , "Koefisien Korelasi Kasus Sembuh" : set_dataFrame2, "Koefisien Korelasi Kasus Meninggal" : set_dataFrame3}

df_final = pd.DataFrame(data=Data)
df_final

,Country,Koefisien Korelasi Penambahan Kasus,Koefisien Korelasi Kasus Sembuh,Koefisien Korelasi Kasus Meninggal
0,Afghanistan,0.954380,0.928012,0.956116
1,Albania,0.916303,0.918440,0.900108
2,Algeria,0.947584,0.944558,0.996659
3,Andorra,0.910058,0.934388,0.800817
4,Angola,0.861227,0.842722,0.877871
5,Antigua and Barbuda,0.949581,0.951123,0.635080
6,Argentina,0.852977,0.792994,0.842842
7,Armenia,0.967169,0.939233,0.959801
8,Australia,0.928616,0.916085,0.804536
9,Austria,0.934142,0.955717,0.858642


In [20]:
# Investigasi mengapa adanya nilai NaN
np.corrcoef([[1,4,5,7,8,9],[0,0,0,0,0,0]])

array([[ 1., nan],
       [nan, nan]])

In [21]:
df_final.describe()

,Koefisien Korelasi Penambahan Kasus,Koefisien Korelasi Kasus Sembuh,Koefisien Korelasi Kasus Meninggal
count,182.000000,182.000000,169.000000
mean,0.838819,0.844243,0.809737
std,0.230792,0.203056,0.242896
min,-0.232133,-0.131105,-0.205843
25%,0.825867,0.831390,0.793317
50%,0.917930,0.911056,0.888168
75%,0.959410,0.950861,0.951760
max,0.996341,0.991231,0.996659


Kesimpulan:

Dari statistika deskriptif diatas, menyatakan bahwa dari sekumpulan sampel data yang diuji, menunjukkan bahwa dari rentang bulan Januari hingga September, rata-rata negara-negara di dunia masih mengalami peningkatan kasus positif seiring bertambahnya hari. Di sisi lain, angka kesembuhan dan meninggal dunia pun mengalami hal yang sama.